<a href="https://www.kaggle.com/code/andrey36912/notebook22e7d99d43?scriptVersionId=194667872" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/telegram-spam-or-ham/dataset.csv


In [8]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [9]:
import pandas as pd

df = pd.read_csv("/kaggle/input/telegram-spam-or-ham/dataset.csv")
df.head()

,text_type,text
0,spam,naturally irresistible your corporate identity...
1,spam,the stock trading gunslinger fanny is merrill ...
2,spam,unbelievable new homes made easy im wanting to...
3,spam,4 color printing special request additional in...
4,spam,do not have money get software cds from here s...


In [ ]:
!pip uninstall torch torchtext
!pip install torch==2.0.1 torchtext==0.15.2




Found existing installation: torch 2.4.0+cpu
Uninstalling torch-2.4.0+cpu:
  Would remove:
    /opt/conda/bin/convert-caffe2-to-onnx
    /opt/conda/bin/convert-onnx-to-caffe2
    /opt/conda/bin/torchrun
    /opt/conda/lib/python3.10/site-packages/functorch/*
    /opt/conda/lib/python3.10/site-packages/torch-2.4.0+cpu.dist-info/*
    /opt/conda/lib/python3.10/site-packages/torch/*
    /opt/conda/lib/python3.10/site-packages/torchgen/*
Proceed (Y/n)? 

In [11]:
!pip show torch torchtext


Name: torch
Version: 2.4.0+cpu
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /opt/conda/lib/python3.10/site-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: accelerate, catalyst, easyocr, fastai, kornia, pytorch-ignite, pytorch-lightning, stable-baselines3, timm, torchaudio, torchmetrics, torchtext, torchvision
---
Name: torchtext
Version: 0.18.0
Summary: Text utilities, models, transforms, and datasets for PyTorch.
Home-page: https://github.com/pytorch/text
Author: PyTorch Text Team
Author-email: packages@pytorch.org
License: BSD
Location: /opt/conda/lib/python3.10/site-packages
Requires: numpy, requests, torch, tqdm
Required-by: 


In [12]:
from torch.utils.data import Dataset
from sklearn.preprocessing import LabelEncoder
import torchtext


class MyDataset(Dataset):
    def __init__(self, X, y, max_len=100):
        self.X = X
        self.y = y
        self.max_len = max_len
        
        self.label_encoder = LabelEncoder().fit(y)
        #self.vocab = torchtext.vocab.GloVe(name='6B', dim=50)
        self.vocab = torchtext.vocab.Vectors("/kaggle/input/glove6b/glove.6B.50d.txt")

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        label = self.label_encoder.transform([self.y.iloc[idx]])
        label = torch.tensor(label)
        
        text = self.X.iloc[idx]
        tokens = text.split()
        
        if len(tokens) > self.max_len:
            tokens = tokens[:self.max_len]
        else:
            diff = self.max_len - len(tokens)
            
            tokens += ['<pad>'] * diff
        
        X = self.vocab.get_vecs_by_tokens(tokens, lower_case_backup=True)
        
        return X, label[0]
    
    
dataset = MyDataset(df['data'], df['labels'])

OSError: /opt/conda/lib/python3.10/site-packages/torchtext/lib/libtorchtext.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs